In [1]:
import llm_core.llm as L
import re

In [8]:
llm = L.LLM('You are a statistic and fact sharing bot.')

s = llm(
    'Hello! Give me a few mathematial constants?',
    response_format={
        'constants': [{'name': str, 'value': float}, ...]
    },
    max_tokens=1000,
    temperature=0,
    verbose=False
)

# for t in s:
#     print(t, end='')

print(s)

{'constants': [{'name': 'pi', 'value': 3.141592653589793}, {'name': 'e', 'value': 2.718281828459045}]}


In [ ]:
llm = L.LLM('You are a statistic and fact sharing bot.')

s = llm(
    'Hello! What are some Wisconsin cities?',
    response_format={'country': str, 'cities': [{'city_name': str, 'population': int}, 10]},
    max_tokens=1000,
    temperature=0,
    verbose=True
)

# for t in s:
#     print(t, end='')

print(s)

In [ ]:
s['cities'][2]['city_name']

In [ ]:
llm._hist

In [ ]:
L.LLM.detokenize(llm._hist_to_prompt())

In [ ]:
import json
import re

class Repr:
    def __init__(self, s): self.s = s
    def __str__(self): return self.s
    def __repr__(self): return self.s

def pretty_response_format(response_format):
    if response_format in [str, int, float]:
        return Repr(response_format.__name__)
    elif type(response_format) == dict:
        return {
            Repr(f'"{k}"'): pretty_response_format(v)
            for k,v in response_format.items()
        }
    elif type(response_format) == list:
        sub, etc = response_format

        if etc == ...:
            return [pretty_response_format(sub), Repr('...')]
        elif type(etc) is int:
            return [pretty_response_format(sub), Repr(f'{etc} total...')]

        else:
            raise Exception(f'List size must be int or `...`, but found: {etc}')

    else:
        raise Exception(f'Unsupported value type: {v}')

# fmt = {'country': str, 'capital': str, 'pop': int}
# fmt = {'country': str, 'capital': str, 'pop': int, 'local_names': [str, ...]}
# fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_country': {'name': str, 'pop': int}}
# fmt = {'country': str, 'capital': str, 'pop': int, 'nearest_cities': [{'name': str, 'pop': int}, 3]}
fmt = str

fmt_prompt = pretty_response_format(fmt)

# print(str(fmt_prompt))
# print(f'{fmt_prompt}')
# assert 0

def gen_response_formated_rec(response_format, prompt_ref, temperature=0, max_tokens=1000):
    if response_format == str:
        prompt_ref[0] += '"'
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=str_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'(^|[^\\])(")', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span(2)[0]]
                break
            prompt_ref[0] += t
        prompt_ref[0] += '"'

    elif response_format in [float, int]:
        s = L.LLM_GLOBAL_INSTANCE(
            prompt_ref[0],
            max_tokens=max_tokens, temperature=temperature,
            stream=True,
            # grammar=num_grammar
        )
        for t in s:
            t = t['choices'][0]['text']
            str_end = re.search(r'[,}]', t)
            if str_end:
                prompt_ref[0] += t[:str_end.span()[0]]
                break
            prompt_ref[0] += t

    elif type(response_format) == dict:
        prompt_ref[0] += '{'
        first = True
        for k,v in response_format.items():
            if not first: prompt_ref[0] += ', '
            first = False
            prompt_ref[0] += f'"{k}": '
            gen_response_formated_rec(v, prompt_ref, temperature, max_tokens)
        prompt_ref[0] += '}'

    elif type(response_format) == list:
        sub_type, etc = response_format

        i = None
        if type(etc) is int:
            i = etc
        else:
            assert etc == ...

        prompt_ref[0] += '['
        while True:
            if not i is None:
                if i == 0: break
                i -= 1

            gen_response_formated_rec(sub_type, prompt_ref, temperature, max_tokens)
            next_tok = L.LLM_GLOBAL_INSTANCE(
                prompt_ref[0],
                max_tokens=1, temperature=0,
                stream=False,
            )['choices'][0]['text'].strip()
            
            if ']' in next_tok:
                break
            elif ',' in next_tok:
                prompt_ref[0] += ', '
                continue
            else:
                raise Exception(f'Theoretically unreachable err: invalid list next tok: {next_tok}')

        prompt_ref[0] += ']'

def gen_response_formated(response_format, prompt, temperature=0, max_tokens=1000):
    json_start = len(prompt)
    prompt_ref = [prompt]
    gen_response_formated_rec(fmt, prompt_ref, temperature, max_tokens)
    return prompt_ref[0][json_start:]

prompt = f'[INST] What is one capital city in Europe? Respond in JSON with this format ending strings with double quotes and numbers with JS notation (commas end rather than separate digits): {fmt_prompt} [/INST] '
print(prompt)

resp = gen_response_formated(fmt, prompt)

print(resp)

# assert 0

# json_start = len(prompt)
# prompt += '{'


# first = True
# for k,v in fmt.items():
#     if not first: prompt += ', '
#     first = False
#     prompt += f'"{k}": '
#     if v == str:
#         prompt += '"'
#         s = L.LLM_GLOBAL_INSTANCE(
#             prompt,
#             max_tokens=1000, temperature=0,
#             stream=True,
#             grammar=str_grammar
#         )
#         for t in s:
#             t = t['choices'][0]['text']
#             str_end = re.search(r'(^|[^\\])(")', t)
#             if str_end:
#                 prompt += t[:str_end.span(2)[0]]
#                 break
#             prompt += t
#         prompt += '"'
#     elif v in [int, float]:
#         s = L.LLM_GLOBAL_INSTANCE(
#             prompt,
#             max_tokens=1000, temperature=0,
#             stream=True,
#             grammar=num_grammar
#         )
#         for t in s:
#             t = t['choices'][0]['text']
#             str_end = re.search(r'[,}]', t)
#             if str_end:
#                 prompt += t[:str_end.span()[0]]
#                 break
#             prompt += t
#     else:
#         raise Exception(f"Unsupported value type: {v}")
# prompt += '}'

# resp = json.loads(prompt[json_start:])

# print(resp)

In [ ]:
try:
    json.loads('owegboieqob')
except json.JSONDecodeError as e:
    raise Exception(f'Cannot parse this json: 23223542')

In [ ]:
llm = L.LLM('You are a Finnish language teacher teaching Finnish to me through conversation. You must give accurate information.')

In [ ]:
s = llm('Oletko koira? respond in English.', response_format='stream', max_tokens=1000, temperature=0.1)
for t in s:
    print(t, end='')

In [ ]:
s = llm('Why did you start your response with an extra space????? AHHHH!!', response_format='stream', max_tokens=1000, temperature=0)
for t in s:
    print(t, end='')

In [ ]:
# llm._hist_to_prompt()
L.LLM.detokenize(llm._hist_to_prompt())
# print(L.LLM.detokenize(llm._hist_to_prompt()))

In [ ]:
llm._hist

In [ ]:
ts = L.LLM_GLOBAL_INSTANCE.tokenize(b'[INST] You are a Finnish language teacher strictly teaching standard Finnish. You must give accurate information. [/INST] Understood.', add_bos=False)
ts = [1, *ts, 2]
ts += L.LLM_GLOBAL_INSTANCE.tokenize(' [INST] What does "mitä on sinun nimi" mean? 1) What is another way to say it? 2) Why is the original version NOT grammatically correct? Tell me in terms of every individual word. [/INST] '.encode('utf-8'), add_bos=False)

In [ ]:

L.LLM.detokenize(ts)

In [ ]:
resp = L.LLM_GLOBAL_INSTANCE(ts, temperature=0, max_tokens=1000, stream=True)
for tok in resp:
    print(tok['choices'][0]['text'], end='')

In [ ]:
resp['choices'][0]['text']

In [ ]:
s = llm('Hello. How are yo?', response_format={'my_name':'...', 'my_response':'...'}, max_tokens=1000)
s

In [ ]:
print(L.LLM.detokenize(llm._hist_to_prompt()))

In [ ]:
$$$  TODO ^ remove the "respond in json" instruction in history

In [ ]:
def logits_processor(prev_tok_ids, next_tok_logits):
    
    # print(prev_tok_ids, next_tok_logits)

    next_tok_logits[995] -= 1000
 
    return next_tok_logits

In [ ]:
# llm = L.LLM('You are a British butler. Start every answer with "you"')
llm = L.LLM('You are a British assistant.')
resp = llm('Hello. What is the capital of Finland?', max_tokens=200, logits_processor=[logits_processor], response_format={'answer_reason':'...', 'answer':'...'})
resp

In [ ]:
print(llm.get_pretty_hist())

In [ ]:
llm.tokenize(' You')

In [ ]:
llm.detokenize([1, 995])